# EDA trafiklabs resrobot API


In [82]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd

In [27]:
load_dotenv()
API_KEY = os.getenv("API_KEY")
url = f"https://api.resrobot.se/v2.1/trip?format=json&originId=740000002&destId=740000003&passlist=true&showPassingPoints=true&accessId={API_KEY}"

response = requests.get(url)
result = response.json()

result.keys()

dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [28]:
result["Trip"]

[{'Origin': {'name': 'Göteborg Centralstation',
   'type': 'ST',
   'id': 'A=1@O=Göteborg Centralstation@X=11973479@Y=57708895@U=1@L=740000002@',
   'extId': '740000002',
   'lon': 11.973479,
   'lat': 57.708895,
   'routeIdx': 0,
   'prognosisType': 'PROGNOSED',
   'time': '11:40:00',
   'date': '2025-01-07',
   'minimumChangeDuration': 'PT15M'},
  'Destination': {'name': 'Malmö Centralstation',
   'type': 'ST',
   'id': 'A=1@O=Malmö Centralstation@X=13000910@Y=55609456@U=1@L=740000003@',
   'extId': '740000003',
   'lon': 13.00091,
   'lat': 55.609456,
   'routeIdx': 12,
   'prognosisType': 'PROGNOSED',
   'time': '14:48:00',
   'date': '2025-01-07',
   'minimumChangeDuration': 'PT15M'},
  'ServiceDays': [{'planningPeriodBegin': '2024-12-11',
    'planningPeriodEnd': '2025-06-15',
    'sDaysR': 'inte varje dag',
    'sDaysI': '7. jan t o m 8. jun 2025; utom 12. t o m 21. apr 2025, 26. apr',
    'sDaysB': 'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFC00F7FFFFFFFFFF000'}],
  'LegList': {'Leg': [{'Or

In [29]:
result["ResultStatus"], result["planRtTs"], result["scrB"]

({'timeDiffCritical': False},
 '1970-01-01T01:00:00.000+01:00',
 '3|OB|MTµ14µ42460µ42460µ42648µ42648µ0µ0µ365µ42433µ1µ0µ1122µ0µ0µ-2147483648µ1µ2|PDHµ378355956619a5b7de5fd8151f63f427|RDµ7012025|RTµ111325|USµ1|RSµINIT')

In [31]:
example_trip = result["Trip"][0]

In [33]:
example_trip.keys()

dict_keys(['Origin', 'Destination', 'ServiceDays', 'LegList', 'calculation', 'TripStatus', 'idx', 'tripId', 'ctxRecon', 'duration', 'rtDuration', 'checksum'])

In [35]:
example_trip["Origin"]

{'name': 'Göteborg Centralstation',
 'type': 'ST',
 'id': 'A=1@O=Göteborg Centralstation@X=11973479@Y=57708895@U=1@L=740000002@',
 'extId': '740000002',
 'lon': 11.973479,
 'lat': 57.708895,
 'routeIdx': 0,
 'prognosisType': 'PROGNOSED',
 'time': '11:40:00',
 'date': '2025-01-07',
 'minimumChangeDuration': 'PT15M'}

In [36]:
example_trip["Destination"]

{'name': 'Malmö Centralstation',
 'type': 'ST',
 'id': 'A=1@O=Malmö Centralstation@X=13000910@Y=55609456@U=1@L=740000003@',
 'extId': '740000003',
 'lon': 13.00091,
 'lat': 55.609456,
 'routeIdx': 12,
 'prognosisType': 'PROGNOSED',
 'time': '14:48:00',
 'date': '2025-01-07',
 'minimumChangeDuration': 'PT15M'}

In [43]:
example_trip["LegList"]["Leg"][0].keys()

dict_keys(['Origin', 'Destination', 'Notes', 'JourneyDetailRef', 'JourneyStatus', 'Product', 'Stops', 'JourneyDetail', 'id', 'idx', 'name', 'number', 'category', 'type', 'reachable', 'waitingState', 'direction', 'directionFlag', 'duration'])

In [62]:
stops = example_trip["LegList"]["Leg"][0]["Stops"]["Stop"]

In [68]:
for stop in stops:
    print(f"{stop['name']}: {stop.get('arrTime')}")

Göteborg Centralstation: None
Mölndal station: 11:48:00
Kungsbacka station: 11:59:00
Varberg station: 12:25:00
Falkenberg station: 12:45:00
Halmstad Centralstation: 13:03:00
Laholm station: 13:25:00
Båstad station: 13:32:00
Ängelholm station: 13:43:00
Helsingborg Centralstation: 14:04:00
Landskrona station: 14:19:00
Lund Centralstation: 14:36:00
Malmö Centralstation: 14:48:00


# Stop lookup

In [73]:
location = "Tranered"


def get_location_id(location: str):
    url = f"https://api.resrobot.se/v2.1/location.name?input={location}&format=json&accessId={API_KEY}"
    response = requests.get(url)
    return response.json().get("stopLocationOrCoordLocation")


tranered = get_location_id(location)

In [74]:
len(tranered)

10

In [76]:
tranered[0].get("StopLocation")

{'productAtStop': [{'icon': {'res': 'prod_gen'}, 'cls': '64'},
  {'icon': {'res': 'prod_gen'}, 'cls': '128'}],
 'timezoneOffset': 60,
 'id': 'A=1@O=Göteborg Tranered@X=11879631@Y=57672516@U=1@L=740025725@B=1@p=1736224604@',
 'extId': '740025725',
 'name': 'Göteborg Tranered',
 'lon': 11.879631,
 'lat': 57.672516,
 'weight': 1931,
 'products': 192,
 'minimumChangeDuration': 'PT0S'}

In [81]:
location_id = 740025725
url = f"https://api.resrobot.se/v2.1/departureBoard?id={location_id}&format=json&accessId={API_KEY}"

response = requests.get(url)

response.json()

{'Departure': [{'JourneyDetailRef': {'ref': '1|167701|28|1|7012025'},
   'JourneyStatus': 'P',
   'ProductAtStop': {'icon': {'res': 'prod_gen'},
    'operatorInfo': {'name': 'Västtrafik',
     'nameS': '279',
     'nameN': '279',
     'nameL': 'Västtrafik',
     'id': '279'},
    'name': 'Länstrafik - Spårväg 11',
    'internalName': 'Länstrafik - Spårväg 11',
    'displayNumber': '11',
    'num': '11',
    'line': '11',
    'lineId': '1279501100001',
    'catOut': 'SLT',
    'catIn': 'SLT',
    'catCode': '6',
    'cls': '64',
    'catOutS': 'SLT',
    'catOutL': 'Länstrafik - Spårväg',
    'operatorCode': '279',
    'operator': 'Västtrafik',
    'admin': '279___',
    'matchId': '11;5011;4947'},
   'Product': [{'icon': {'res': 'prod_gen'},
     'operatorInfo': {'name': 'Västtrafik',
      'nameS': '279',
      'nameN': '279',
      'nameL': 'Västtrafik',
      'id': '279'},
     'name': 'Länstrafik - Spårväg 11',
     'internalName': 'Länstrafik - Spårväg 11',
     'displayNumber': '

In [85]:
pd.DataFrame(response.json()["Departure"])

,JourneyDetailRef,JourneyStatus,ProductAtStop,Product,Notes,name,type,stop,stopid,stopExtId,lon,lat,time,date,reachable,direction,directionFlag
0,{'ref': '1|167701|28|1|7012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 11,ST,Göteborg Tranered,A=1@O=Göteborg Tranered@X=11879631@Y=57672516@...,740025725,11.879631,57.672516,11:44:00,2025-01-07,True,Bergsjön Komettorget (Göteborg kn),1
1,{'ref': '1|168013|16|1|7012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 11,ST,Göteborg Tranered,A=1@O=Göteborg Tranered@X=11879631@Y=57672516@...,740025725,11.879631,57.672516,11:46:00,2025-01-07,True,Göteborg Saltholmens Brygga,2
2,{'ref': '1|167701|29|1|7012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 11,ST,Göteborg Tranered,A=1@O=Göteborg Tranered@X=11879631@Y=57672516@...,740025725,11.879631,57.672516,11:54:00,2025-01-07,True,Bergsjön Komettorget (Göteborg kn),1
3,{'ref': '1|168013|17|1|7012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 11,ST,Göteborg Tranered,A=1@O=Göteborg Tranered@X=11879631@Y=57672516@...,740025725,11.879631,57.672516,11:56:00,2025-01-07,True,Göteborg Saltholmens Brygga,2
4,{'ref': '1|167701|30|1|7012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 11,ST,Göteborg Tranered,A=1@O=Göteborg Tranered@X=11879631@Y=57672516@...,740025725,11.879631,57.672516,12:04:00,2025-01-07,True,Bergsjön Komettorget (Göteborg kn),1
5,{'ref': '1|168013|18|1|7012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 11,ST,Göteborg Tranered,A=1@O=Göteborg Tranered@X=11879631@Y=57672516@...,740025725,11.879631,57.672516,12:06:00,2025-01-07,True,Göteborg Saltholmens Brygga,2
6,{'ref': '1|167701|31|1|7012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 11,ST,Göteborg Tranered,A=1@O=Göteborg Tranered@X=11879631@Y=57672516@...,740025725,11.879631,57.672516,12:14:00,2025-01-07,True,Bergsjön Komettorget (Göteborg kn),1
7,{'ref': '1|168013|19|1|7012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 11,ST,Göteborg Tranered,A=1@O=Göteborg Tranered@X=11879631@Y=57672516@...,740025725,11.879631,57.672516,12:16:00,2025-01-07,True,Göteborg Saltholmens Brygga,2
8,{'ref': '1|167701|32|1|7012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 11,ST,Göteborg Tranered,A=1@O=Göteborg Tranered@X=11879631@Y=57672516@...,740025725,11.879631,57.672516,12:24:00,2025-01-07,True,Bergsjön Komettorget (Göteborg kn),1
9,{'ref': '1|168013|20|1|7012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 11,ST,Göteborg Tranered,A=1@O=Göteborg Tranered@X=11879631@Y=57672516@...,740025725,11.879631,57.672516,12:26:00,2025-01-07,True,Göteborg Saltholmens Brygga,2
